Análisis del despliegue de discursos discriminatorios y de odio durante la pandemia en TWITTER

LINK A FUENTE

LINK A ARTICULO

In [1]:

import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Replace 'your_file_path' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/2. Data-work/python-space/hate_speech/df_tweets_arg.csv'



Mounted at /content/drive


Tw_arg tiene ya solo los medios de Argentina

'LANACION',
'clarincom',
'cronica',
'infobae',
'izquierdadiario',
'laderechadiario',
'laderechamedios',
'pagina12',
'perfilcom'

In [3]:
tw_arg = pd.read_csv(file_path)

In [4]:
diarios = ['LANACION',
'clarincom',
'cronica',
'infobae',
'izquierdadiario',
'laderechadiario',
'laderechamedios',
'pagina12',
'perfilcom']

In [7]:
tw_arg.head()

,tweet_id_noticia,user_noticia,date_noticia,title_noticia,url_noticia,tweet,date_tweet,APPEARANCE,CLASS,CRIMINAL,DISABLED,LGBTI,POLITICS,RACISM,WOMEN,CALLS
0,1376940813968609288,clarincom,2021-03-30 16:54:00.700,Segunda ola de coronavirus: preocupan las reun...,https://www.clarin.com/politica/segunda-ola-co...,@clarincom A mi me preocupa el trabajo.. La ev...,2021-03-30 17:03:00.900,0,0,0,0,0,0,0,0,0
1,1376940813968609288,clarincom,2021-03-30 16:54:00.700,Segunda ola de coronavirus: preocupan las reun...,https://www.clarin.com/politica/segunda-ola-co...,@clarincom Lo que preocupa. https://t.co/Vmf9V...,2021-03-30 17:05:04.500,0,0,0,0,0,0,0,0,0
2,1376940813968609288,clarincom,2021-03-30 16:54:00.700,Segunda ola de coronavirus: preocupan las reun...,https://www.clarin.com/politica/segunda-ola-co...,@clarincom Lo que les preocupa. https://t.co/P...,2021-03-30 17:06:03.100,0,0,0,0,0,0,0,0,0
3,1376940813968609288,clarincom,2021-03-30 16:54:00.700,Segunda ola de coronavirus: preocupan las reun...,https://www.clarin.com/politica/segunda-ola-co...,@clarincom Le recomendaríamos al presidente de...,2021-03-30 17:11:02.300,0,0,0,0,0,0,0,0,0
4,1376940813968609288,clarincom,2021-03-30 16:54:00.700,Segunda ola de coronavirus: preocupan las reun...,https://www.clarin.com/politica/segunda-ola-co...,@clarincom Para salvar al correo de la quiebra...,2021-03-30 17:26:00.600,0,0,0,0,0,0,0,0,0


In [9]:
tw_arg['date_tweet'] = pd.to_datetime(tw_arg['date_tweet'])
tw_arg['date_only'] = tw_arg['date_tweet'].dt.date
daily_counts = tw_arg.groupby('date_only').size().reset_index(name='tweet_count')
daily_counts

,date_only,tweet_count
0,2020-02-10,4513
1,2020-02-11,2407
2,2020-02-12,1815
3,2020-02-13,4446
4,2020-02-14,5992
...,...,...
496,2021-06-21,5709
497,2021-06-22,3501
498,2021-06-23,52
499,2021-06-24,51


In [8]:
# prompt: tw_arg contar la cantidad de tweets por día date_tweet

# Convert 'date_tweet' column to datetime objects if it's not already
tw_arg['date_tweet'] = pd.to_datetime(tw_arg['date_tweet'])

# Group by date and count the number of tweets
tweets_per_day = tw_arg.groupby('date_tweet')['date_tweet'].count()

# Print or use the result
tweets_per_day

,date_tweet
date_tweet,
2020-02-10 00:04:02.500,1
2020-02-10 00:04:02.900,1
2020-02-10 00:05:00.500,1
2020-02-10 00:05:01.300,1
2020-02-10 00:06:02.500,1
...,...
2021-06-25 04:10:05.800,1
2021-06-25 04:35:03.000,1
2021-06-25 06:32:00.700,1
